In [13]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [14]:
small_train_data_folder = os.getcwd() + '/data/train_sm'
small_test_data_folder = os.getcwd() + '/data/test_sm'

In [15]:
first = plt.imread('./data/train_sm/set175_1.jpeg')
dims = np.shape(first)
print(dims)

(2329, 3100, 3)
